In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset,Dataset
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve
#Class to load the dataset images from drivce
import os
import cv2
import numpy as np

from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras


In [2]:
#for unpacking data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin')
    return dict

In [3]:
#for loading data
def load_batch_file(batch_filename):
    filepath = os.path.join('./cifar-10-batches-py/', batch_filename)
    unpickled = unpickle(filepath)
    return unpickled

In [4]:
batch_size = 32  # for updation of weights in a batch.
num_classes = 10  # Number of class for the dataset
epochs = 3

In [11]:
train_batch_1 = load_batch_file('./data_batch_1')
train_batch_2 = load_batch_file('./data_batch_2')
train_batch_3 = load_batch_file('./data_batch_3')
train_batch_4 = load_batch_file('./data_batch_4')
train_batch_5 = load_batch_file('./data_batch_5')
test_batch = load_batch_file('./test_batch')

x_train = np.concatenate([train_batch_1['data'],train_batch_2['data'],train_batch_3['data'],train_batch_4['data'],train_batch_5['data']])
y_train = np.concatenate([train_batch_1['labels'],train_batch_2['labels'],train_batch_3['labels'],train_batch_4['labels'],train_batch_5['labels']])
x_test = test_batch['data']
y_test = test_batch['labels']

le = preprocessing.LabelEncoder()
y_test = le.fit_transform(y_test)

#reshaping data for convolutional neural netowrks
x_train = x_train.reshape(50000,32,32,3) 
y_train = y_train.reshape(50000,1)
x_test = x_test.reshape(10000,32,32,3)
y_test = y_test.reshape(10000,1)

#converting data to float so we can divide by 255(UNICODE)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#categorically converting labels to become binary
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [12]:
print(x_train)
print('------------------------------')
print(y_train)
print('------------------------------')
print(x_test)
print('------------------------------')
print(y_test)


[[ 59  43  50 ... 140  84  72]
 [154 126 105 ... 139 142 144]
 [255 253 253 ...  83  83  84]
 ...
 [ 35  40  42 ...  77  66  50]
 [189 186 185 ... 169 171 171]
 [229 236 234 ... 173 162 161]]
------------------------------
[6 9 9 ... 9 1 1]
------------------------------
[[158 159 165 ... 124 129 110]
 [235 231 232 ... 178 191 199]
 [158 158 139 ...   8   3   7]
 ...
 [ 20  19  15 ...  50  53  47]
 [ 25  15  23 ...  80  81  80]
 [ 73  98  99 ...  94  58  26]]
------------------------------
[3 8 8 ... 5 1 7]


In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:])) 
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #getting max values from matrices
model.add(Dropout(0.25)) #dropping weights

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) #getting max values from matrices
model.add(Dropout(0.25))

model.add(Flatten()) #flattening data
model.add(Dense(512)) # connecting neurons deeply
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [15]:
#decay = regularization
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test, y_test),shuffle=True)

Epoch 1/3
1563/1563 [==============================] - 227s 143ms/step - loss: 2.1834 - accuracy: 0.1539 - val_loss: 2.1141 - val_accuracy: 0.1627
Epoch 2/3
1563/1563 [==============================] - 208s 133ms/step - loss: 1.9723 - accuracy: 0.2200 - val_loss: 1.8788 - val_accuracy: 0.2596
Epoch 3/3
1563/1563 [==============================] - 202s 130ms/step - loss: 1.8635 - accuracy: 0.2588 - val_loss: 1.8322 - val_accuracy: 0.3013


In [17]:
#evaluating 
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 10s 31ms/step - loss: 1.8322 - accuracy: 0.3013
Test loss: 1.8322426080703735
Test accuracy: 0.3012999892234802


In [18]:
#predicting
pred = model.predict(x_test)

313/313 [==============================] - 10s 30ms/step


In [19]:
#making predictions 1D for accuracy and report
pred = np.argmax(pred, axis=1)

In [20]:
#making testing labels 1D for accuracy and report
y_test = np.argmax(y_test,axis=1)

In [24]:
#accuracy
accuracy = accuracy_score(y_test, pred) 

In [25]:
accuracy

0.3013

In [22]:
from sklearn.metrics import classification_report

report = classification_report(y_test, pred)

C:\Users\Sebastain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sebastain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sebastain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
print(report)

              precision    recall  f1-score   support

           0       0.31      0.20      0.24      1000
           1       0.31      0.36      0.33      1000
           2       0.27      0.23      0.25      1000
           3       0.00      0.00      0.00      1000
           4       0.22      0.08      0.12      1000
           5       0.23      0.43      0.30      1000
           6       0.21      0.04      0.06      1000
           7       0.26      0.39      0.31      1000
           8       0.40      0.72      0.52      1000
           9       0.36      0.56      0.44      1000

    accuracy                           0.30     10000
   macro avg       0.26      0.30      0.26     10000
weighted avg       0.26      0.30      0.26     10000

